# Whisper Clustering for Topic

In [1]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP
from loguru import logger
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from transformers.pipelines.base import KeyDataset
from transformers import BertTokenizerFast, AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoModelForSequenceClassification
import pandas as pd

In [2]:
# load in data

YT_or_TT = 'TT'

if YT_or_TT == 'YT':
    df = pd.read_parquet('/data/inet-large-scale-twitter-diffusion/ball4321/data_c/YT/audio_transcribed.parquet')
elif YT_or_TT == 'TT':
    df = pd.read_parquet('/data/inet-large-scale-twitter-diffusion/ball4321/data_c/TikTok/01_raw/videos_transcribed.parquet')

In [19]:
len(df)

5685

In [3]:
(df['text'].apply(lambda x: any([i in x for i in ['metoo', 'MeToo']]))).sum()

2

In [15]:
df['description_lang'].value_counts()

description_lang
un    862
en    617
es    564
he    483
it    478
     ... 
sn      1
ga      1
cy      1
sl      1
fy      1
Name: count, Length: 69, dtype: int64

In [21]:
df[df['description_lang']=='en'][20:30]

,video_id,author_id,description,description_lang,text
414,7304948664215080194,7053449319785595931,Such as cats and dogs🤣🤣🤣 #songjihyo #mongji #mongjihyo #mongjis #leekwangsoo #kwangmong #kwangmongsibling,en,뭐야?. 여기요?. 저기요. 누구야?. 오빠 진짜 아빠 죄송해요. 감정이요?. 누가 지금 내동같은지?. 내동같은 사람 누구냐고. 아니 누구신데 오자마자. 저기요. 누구냐고. 저요?. 아빠 진짜 아빠 죄송해요. 손지효씨. 오자마자 이렇게 저기. 싸대기를 날리면 어떡해. 내가. 선수입료달라고 해서 먼저 줬더니. 먹튀한 놈 누구야?. 아니 이거 아님. 아 내가 바람 난 남친. 정수로 잘달라고는 너무 창피해서. 온라인으로. 오랬던. 돈가족 한 먹튀를 해?. 아니 근데 오자마자. 아니 우리는 디지털 그쪽으로는 아님. 아 신기 너야?. 아니 아니 저 온라인 못해요. 아니 누구신데. 먹튀 빨리 돈 내놔. 니가 앉아 앉아. 앉아 앉아
420,7304270656621923589,7175145736278787098,my heewon🤭 #heewon #jungwon #heeseung #enhypen #tiktok #foryoupage #foryoupage #fypシ゚viral #fypシ゚viral #fypシ゚viral #foryoupage,en,Hey!
513,7288250378368339246,7264943955849462827,[Pokemon] Snow Girl’s Guest #funny #relax #interesting #fyp,en,다음 영상에서 또 만나요!
522,7286879281777478918,7051572159965758469,nome:Academy’s Undercover Professor chapter 67 #manhwareccomendation #webtoonrecommendation #manhwaedit #webtoonedit #manhwa #webtoon #manhua #edit #foryou #fly #spolers #academysundercoverprofessor,en,I'll be with you from dawn till dawn. But you'll never be alone. I'll be with you from dawn till dawn. I'll be with you from dawn till dawn. Baby I'm right here
531,7285394132565069057,7253100045724042245,Spoiler Manager Kim Chapter 106!!!#manhwa#managerkim#spoiler,en,"I love you, son.. And you shattered my soul.. Cause you were calling, I'm a fool."
533,7284849085754985733,7160002924150555650,Manhwa: Academy's Under Cover Professor #아카데미에위장취업당했다 #undercoverprofessor #undercoverprofessor #academysundercoverprofessor #ludgerchelysie #manhwa #manhwareccomendation #manhwaedit #CapCut,en,"Please don't let it show. Oh baby. I don't wanna know. I took my clothes and took my love. And one night I got a new spot. I left the party with the party. Marking marks on the floor. She calls my stoner, but I told her I'm a loner. She likes my watch and my drop top. And like her stoner ball"
539,7284262878557900033,7276848508266955782,Part 7 #movieclips #movies #homcoming #clips #foryou #us,en,
553,7280839684035529992,7096067550131782658,catch evil5화!#캡컷 #즈상황극 #홍박사챌린지 #로블록스 #추천추천추천추천추천추천추천추천추천추천 #추천추천추천추천추천추천추천추천추천추천,en,얘들아 어떻게 됐어 우린 잡았어. 우리도 잡았어 이게 바로 물약의 힘인가 봐. 그런 것 같다. 그 난 몸살기가 좀 있는 것 같아서. 먼저 들어가서 쉴게. 소라가 왜 그러지. 내가 한번 따라가 볼게. 소라야 왜 그래. 펠레나에 대한 생각 때문에. 만약 펠레나랑 싸우게 되는 날이 생긴다면. 흥! 그 전에 돌려놔야지. 한편 뱀파이어가. 펠레나 너 그거 들었어?. 뭔데 호들갑이야. 아니 카를로스랑 그 빨갱이 걔네들이랑 당했다던데. 뭐?. 하 그럼 이제 우리도 슬슬 준비해야지. 야 자비에를 불러와. 아씨 왜 부르는데. 어 펠레나 있었네. 아 됐고 너 메이시랑 가서 걔들 좀 어떻게 해봐. 펠레나 부탁이라면 해야지. 야 너 나한텐 그렇게 짜증내고 펠레나가 그렇게 좋냐. 당연하지 너 같은 거랑은 비교도 안 되겠군. 아 진짜 내가 자비에를 좋아하는데. 쟤는 눈치 없이 헬러나만 좋아하고. 아 둘 다 닥치고 얼른 가라. 주변 뱀파이어 발견. 일단 나가자. 헐 원해 우리나라?. 그러니까. 아 스카 등급은?. 3등급 3등급. 하지만 그냥 3등급은 아니고. 펠레나에 수련을 받은 자들. 뭐?. 펠레나 일단 가자. 근데 펠레나 수련을 받은 자들이면. 두 명일 텐데. 저번처럼 나 놓을까?. 시간도 없으니 그게 좋겠다. 헐 여기 좀 충격적인 비주얼인데?. 그러니까. 우 등골이 오싹하네. 안녕 헐 너무 반가워 아니. 나 계속 너네 기다렸잖아 예. 우린 너 같은 곳들이 사라지면 좋겠는데?. 이얍. 아씨 이거 안 치워?. 우리는 너희들에게 항상 말하잖아. 방심하지 말라고. 너희들의 지능이 딸리는 거. 뭐야 말조심해. 너희 같은 얘들한테는 그럴 필요 없어. 넌 이런 애들 하나 상대못하고. 말싸움이나 하고 있냐?. 아 헬레나 미안. 뭐?. 헬레나?
555,7280534273755663622,7160002924150555650,Hans VS Ludger #ludgerchelysie #Undercoverprofessor #academysundercoverprofessor #manhwa #manhwareccomendation #아카데미에위장취업당했다 #manhwa #webtoon,en,
704,7293872833585859882,7178503672023188522,🥰❤️ #יהודהורבורג #שניורבורג #אדיטים #אדיט #ויראלי #פוריו #פוריוישראל #בעלתהחלומות #בעלתהחלומות2 #החולמים #oursong.edits,en,Byeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee


In [24]:
type(df['description'].str.lower())

pandas.core.series.Series

In [31]:
df[df['description'].apply(lambda x: 'metoo' in x.lower())][:30]

,video_id,author_id,description,description_lang,text
107,7308748333248466222,7308715153661789227,▶▶▶▶▶▶add me▶▶▶▶▶▶@amandajordan5 @nominomi891 @adrianita0512 @sj_maik @ramrez.shaira55 @_jennymoreno @adrianaalejandrag5 @johandanniel2 @mques21 @mario_escobar71 @albenypatriciapos @daniellopezdurang @dy2ctwqa53lw @emiaurora2964 @germansanchez9619 @hetero3671 @user5381704437742 @-pipelipe170 @temasos_letras @johafihqn35 @.ladane @iam.saray._ @karenanayaayola @alejacontreras6 @jorgenarvaez58 @joan_florez @julian_19383737 @mariajoseospinavi8 @derlismartinez1308 @karenpaoruiz @saoryfashion @user135993226527 @oscar.bolvar2 @dueimaragudelo186 @franciiscoduque @agroquirama @maripau_55 @stardust_w11 @krisguerra3 @leonard_f.a @kellygonzalez174 @sachez052 @yordang29 @ramos64142 @yulsinis @yula2314 @zerjiio @mavilez_111 @santiagogomezroja0 @santiagogmezrojas4 @sim0n._prb @lauradiaz26081 @claudiasilvasaray @theboniceman @naury_daniela @adlejandr @avpa007 @eithan496 @tistinap @nloveee @luu.nuuu @dluchoparrilla @elchiirri @-sandralara2681 @felipe.nieto01 @eider.garcia24 @dalilitaisabel @cieloortiz65 @nanisalas17 @carolinaperdomo934 @omaragarciac @hayverstiven @--metooido @johannacnavarrom @yeffersonf15 @sil_sasgu @leslygutierrez_05 @sandraprb1097 @libardogutierrez @gregoriocalderon44 @ximena_2806_ @mercylopez511 @hipnoterapeuta_mao @yotambiensoymerengue @yurhiipalacios @saraperezsanches @thomasvalenciague5 @esneiderperez6975 @mauriciodiaz.21 @torito656 @sofiiaa630 @endrymiichel83 @juliluna119 @michell_ossa1 @jhojanpiedrahita17 @juanpabloperezper3,en,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប �
172,7293250197369490694,7289190245006754821,Quiero invitar a los expertos a analizar los videos que he compartido sobre lo que nos hicieron a mi bebé y a mi. Conoce y alcanza la verdad Denuncia ahora! #cuentasconmigo #cuentalo #niunamas #niunamenos #metoo #yotambien #violenciadegenero #noalmaltratoinfantil #secuestro #justicia,es,"Hola a todos, buenas noches. Quiero hacer una invitación muy especial para ahogados, psicólogos, psiquiatras y especialistas del análisis de la comunicación no verbal. para que puedan compartirnos lo que ustedes pueden percibir acerca del comportamiento de todas las personas involucradas en el caso, en mi historia.. Y bueno, invito nuevamente a los padres y madres víctimas del sistema para que se unan a mi movimiento Númeras Cuentas Conmigo."
181,7291072901124508933,7289190245006754821,¡Únete a este movimiento! #cuentasconmigo #cuentalo #yotambien #niunamenos #niunamas💜 #metoo,es,"Hola a todos, mi nombre es María Alejandra Pérez Ruiz, tengo 36 años, soy madre soltera,. soy madre de una bella niña, su nombre es Lucía Pérez, estoy aquí para contar mi. historia y de esta manera poder animarlos a ustedes, padres y madres, a que cuenten. su historia. Quiero que conozcamos y alcancemos la verdad, denuncia ahora, únete a mí. con el numeral Cuentas con mí."
426,7303402953694563585,6619742210534424577,#미투챌린지 #metoo #metoocallenge #추천 #추천떠라 #추천뜨게해주세요,un,"If I was you, I'd wanna be me too. I'd wanna be me too. I'd wanna be me too.. If I was you, I'd wanna be me too. I'd wanna be me too."
478,7295720512423021832,7114487268307633154,자존감 떨어질때 이 애기보고 업시키기😆💖 - - 🎵Meghan Trainor - Me Too ✔️출처 : tiktok @hausofbroox - - #메간트레이너 #mechantrainor #metoo #노래추천 #팝송추천 #popsong #띵곡추천 #cover #coversongs #live #concert #kpop,ko,Who's that sexy thing I see over there That's me standing in the mirror. What's that icy thing hanging round my neck
2094,7305305172429884678,7251267524356146177,九月，我寫給自己的一封信 成為空中服務生服我喜歡，在空中刷馬桶沒關係 特別回頭看過去5年的自己 有哭有笑、有血有汗、有失敗 也有朋友一起奮鬥 有人自己為自己努力遠走他鄉 現在 剛好在中間的歲數 我什麼都沒有，只有那麼一點點的自由 我常在心裡擔心身邊的人好不好？ 卻忘記了問自己．你還好嗎？ 9月是我討厭的月份， 改變 選擇 決定 最多的時間 正也是 我不喜歡自己出生的時候 為什麼一定要喜歡自己？ 每本書都在寫如何愛自己，別人才會愛你 做自己就能有快樂… 那些話不自負嗎？簡單不負責任 \t 某天林志玲與蔡康永對話， 蔡康永曾經說過「厲害的人都很累」， 她審視及思考蔡康永說的話很久得出來的結論。 林志玲說自己沒有辦法 去「做自己」， 因為他們一直保持自律能力，學習控管自己情緒。 無時無刻警覺周遭情況，不隨意懈怠， 不將自己的情緒發洩在別人身上

In [ ]:
filter_by_cluster(
    ,
    ,
    embedding_model = '/data/inet-large-scale-twitter-diffusion/ball4321/data_b/04_models/LaBSE',

)

In [32]:
embedding_model = SentenceTransformer('/data/inet-large-scale-twitter-diffusion/ball4321/data_b/04_models/LaBSE')

# Step 2 - Reduce dimensionality.
umap_model = UMAP(
    n_neighbors=10,
    n_components=5,
    min_dist=0.0,
    metric='cosine',
    random_state=42,
    verbose=True
)

# Step 3 - Cluster reduced embeddings.
dbscan_model = HDBSCAN(
    min_cluster_size=10,
    # min_samples=50,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

# Step 4 - Tokenize topics.
vectorizer_model = CountVectorizer(
    stop_words="english"
)

# Step 5 - Create topic representation.
ctfidf_model = ClassTfidfTransformer(
    seed_words = ['#MeToo', 'metoo', 'MeToo', 'sexual', 'assault', 'niunamenos']
)

In [36]:
topic_model = BERTopic(
    language='multilingual', # Set to 'multilingual' for datasets with languages other than English. N.B. documentation says this is not used if embedding_model is provided
    top_n_words=10,
    n_gram_range=(1, 3),
    # min_topic_size=10,
    nr_topics='50',
    low_memory=False,
    calculate_probabilities=False, # The probabilities of all topics per document. Might need to set to false for gpu https://github.com/rapidsai/cuml/issues/5127. see also https://github.com/rapidsai/cuml/issues/4879
    seed_topic_list=None, # Like CorEx
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=dbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    verbose=True
)

In [33]:
df.columns

Index(['video_id', 'author_id', 'description', 'description_lang', 'text'], dtype='object')

In [34]:
df[1000:1100]['text']

1000     è un film che mi ronzai in testa da un bel po'. Ho pensato molto se fare questo video, se.  fosse giusto offrire una chiave di lettura dei femenicidi di cui tanto si sta parlando.  in questi giorni attraverso ciò di cui mi occupo, attraverso il cinema. Si intitola.  Man, vede la partecipazione di A24 e fin dal titolo ragiona su quello slogan insensato.  che è Not All Man, non tutti gli uomini, che così spesso usiamo per tirarci fuori..  Lo fa togliendo quel Not All dal titolo e offrendosi come una grande autoaccusa.  a tutti i comportamenti più o meno consapevoli che ci rendono tutti i responsabili del fenomeno,.  tutti i parti del problema. La protagonista, Jesse Buckley, è una vedova che si trasferisce.  in campagna per elaborare il lutto del marito e non solo lei non ne ha alcuna colpa, ma.  il gesto estremo del marito, seguito a un litigio, è stato solo l'ennesimo, estremo.  tentativo di esercitare un controllo su di lei, di farla sentire in colpa per.  colpe che in realtà

In [37]:
docs = df[df['description_lang']=='en']['text']
out = topic_model.fit_transform(docs)

2024-01-29 16:39:00,071 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

2024-01-29 16:40:30,453 - BERTopic - Embedding - Completed ✓
2024-01-29 16:40:30,455 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_components=5, n_neighbors=10, random_state=42, verbose=True)
Mon Jan 29 16:40:30 2024 Construct fuzzy simplicial set
Mon Jan 29 16:40:31 2024 Finding Nearest Neighbors
Mon Jan 29 16:40:34 2024 Finished Nearest Neighbor Search
Mon Jan 29 16:40:35 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs


2024-01-29 16:40:38,001 - BERTopic - Dimensionality - Completed ✓
2024-01-29 16:40:38,002 - BERTopic - Cluster - Start clustering the reduced embeddings


Mon Jan 29 16:40:37 2024 Finished embedding


2024-01-29 16:40:38,261 - BERTopic - Cluster - Completed ✓
2024-01-29 16:40:38,264 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-29 16:40:38,408 - BERTopic - Representation - Completed ✓
2024-01-29 16:40:38,410 - BERTopic - Topic reduction - Reducing number of topics
2024-01-29 16:40:38,517 - BERTopic - Topic reduction - Reduced number of topics from 13 to 8


In [38]:
topic_model.get_topic_info()

Topic  Count                                           Name  \
0     -1    141                             -1_उसक_रहन_पड_just   
1      0    298                             0_oh_like_don_lali   
2      1     58                                          1____   
3      2     38                   2_watching_thanks_매콤하고_짭조름해요   
4      3     30                             3_hey_mean_like_oh   
5      4     24                                4_música_feel__   
6      5     15                     5_halloween_kill_boss_home   
7      6     13  6_বব_បប_byeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee_   

                                                               Representation  \
0                     [उसक, रहन, पड, just, ooh, people, way, need, know, say]   
1                        [oh, like, don, lali, know, just, 노래는, 아아, ciao, ہے]   
2                                                        [, , , , , , , , , ]   
3  [watching, thanks, 매콤하고, 짭조름해요, 짜장면은, thank, 감사합니다, 시청해주셔서, subscribe, 구독]   
4        [hey, mean, like, oh, tchau, それとも, yeah, think, 也许是, ahhhhhhhhhhhhh]   
5                                              [música, feel, , , , , , , , ]   
6         [halloween, kill, boss, home, cosplayer, kiss, nuh, fool, love, uh]   
7                [বব, បប, byeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee, , , , , , , ]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
topic_model.visualize_topics()

In [ ]:
docs = df['videoTitle']
hierarchical_topics = topic_model.hierarchical_topics(docs)

In [ ]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)